In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime

In [9]:
def xnpv(rate, cashflows):

    chron_order = sorted(cashflows, key=lambda x: x[0])
    t0 = chron_order[0][0]  # t0 is the date of the first cash flow
    return sum([cf / (1 + rate) ** ((t - t0).days / 365.0) for (t, cf) in chron_order])

def xirr(transactions):
    years = [(ta[0] - transactions[0][0]).days / 365.0 for ta in transactions]
    residual = 1
    step = 0.05
    guess = 0.1
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, ta in enumerate(transactions):
            try:
                residual += ta[1] / pow(guess, years[i])
            except ZeroDivisionError as e:
                return 0
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return guess - 1

In [4]:
df=pd.read_csv("xnpv_xirr_sample.csv")
df.set_index(['security'])
df.head(5)

,security,date,Cash_Flows
0,A,10/1/2021,-360314.736900
1,A,11/15/2021,7105.915202
2,A,12/15/2021,8199.971438
3,A,1/15/2022,10245.763650
4,A,2/15/2022,11352.041900


In [7]:
df['date'] = pd.to_datetime(df['date'])
cf_list = df['Cash_Flows'].tolist()
dates_list = df['date'].tolist()
s = pd.Series(tuple(zip(dates_list, cf_list)), index=df.index)
cf = s.groupby(df['security']).apply(tuple).reset_index(name='cashflows')
cf.head(5)

,security,cashflows
0,A,"((2021-10-01 00:00:00, -360314.7369), (2021-11..."
1,B,"((2021-10-01 00:00:00, -513101.7782), (2021-11..."
2,C,"((2021-10-01 00:00:00, -217588.1772), (2021-11..."


In [11]:
cf['XIRR'] = cf.apply(lambda x:xirr(x['cashflows']), axis=1)
cf['XNPV'] = cf.apply(lambda x:xnpv(0.04, x['cashflows']), axis=1)
cf.head(5)

,security,cashflows,XIRR,XNPV
0,A,"((2021-10-01 00:00:00, -360314.7369), (2021-11...",0.032151,-3533.034777
1,B,"((2021-10-01 00:00:00, -513101.7782), (2021-11...",-0.006875,-23712.186159
2,C,"((2021-10-01 00:00:00, -217588.1772), (2021-11...",0.002931,-17853.346198
